SYMBOL     FILES    FIRST     LAST
ARBUSDT       25   202310   202510
AVAXUSDT      25   202310   202510
BNBUSDT       25   202310   202510
BTCUSDT       25   202310   202510
ETHUSDT       25   202310   202510
MATICUSDT     12   202310   202409
SOLUSDT       25   202310   202510
SUIUSDT       25   202310   202510
TRXUSDT       25   202310   202510


In [4]:
# 1) Tải bổ sung 1m cho MATIC
try:
    from data_minute import download_minute_range
except Exception:
    import sys, os
    sys.path.append("simplest_ml/w1")
    from data_minute import download_minute_range

minute_dir = "data/minute"  # đổi thành "data/minute" nếu chạy trong w1/
download_minute_range(
    "MATICUSDT",
    start="2024-10-01",
    end="2025-10-30",
    out_dir=minute_dir,
    interval="1m",
    overwrite=False,
    sleep=0.25,
    verbose=True
)



[fetch] MATICUSDT 1m 202410: 2024-10-01 → 2024-11-01
  -> no data for 202410 (empty month)
[fetch] MATICUSDT 1m 202411: 2024-11-01 → 2024-12-01
  -> no data for 202411 (empty month)
[fetch] MATICUSDT 1m 202412: 2024-12-01 → 2025-01-01
  -> no data for 202412 (empty month)
[fetch] MATICUSDT 1m 202501: 2025-01-01 → 2025-02-01
  -> no data for 202501 (empty month)
[fetch] MATICUSDT 1m 202502: 2025-02-01 → 2025-03-01
  -> no data for 202502 (empty month)
[fetch] MATICUSDT 1m 202503: 2025-03-01 → 2025-04-01
  -> no data for 202503 (empty month)
[fetch] MATICUSDT 1m 202504: 2025-04-01 → 2025-05-01
  -> no data for 202504 (empty month)
[fetch] MATICUSDT 1m 202505: 2025-05-01 → 2025-06-01
  -> no data for 202505 (empty month)
[fetch] MATICUSDT 1m 202506: 2025-06-01 → 2025-07-01
  -> no data for 202506 (empty month)
[fetch] MATICUSDT 1m 202507: 2025-07-01 → 2025-08-01
  -> no data for 202507 (empty month)
[fetch] MATICUSDT 1m 202508: 2025-08-01 → 2025-09-01
  -> no data for 202508 (empty month)

[]

Missing for BTCUSDT: []


In [ ]:
# 1) Import the downloader (robust to different working dirs)
try:
    from data_minute import download_minute_for_symbols
except Exception:
    import sys, os
    sys.path.append("simplest_ml/w1")
    from data_minute import download_minute_for_symbols

# 2) Resolve output directory relative to current notebook CWD
import os
candidates = [
    "data/minute",                 # when running inside simplest_ml/w1
    # "simplest_ml/w1/data/minute",  # when running from repo root
]
out_dir = None
for p in candidates:
    try:
        os.makedirs(p, exist_ok=True)
        out_dir = p
        break
    except Exception:
        pass
if out_dir is None:
    raise RuntimeError("Cannot create output dir in any candidate path.")

print(f"Saving 1m shards to: {out_dir}")

# 3) Configure symbols and date range
symbols = [
    "BTCUSDT","BNBUSDT","SOLUSDT","ETHUSDT","AVAXUSDT",
    "TRXUSDT","ARBUSDT","SUIUSDT"
]
start = "2023-10-30"   # inclusive
end   = "2025-10-30"   # exclusive (API window end)

# 4) Download (prints per-month progress; safe to re-run)
download_minute_for_symbols(
    symbols,
    start=start,
    end=end,
    out_dir=out_dir,
    interval="1m",
    overwrite=False,   # set True to re-pull existing months
    sleep=0.25,        # be polite to API, reduce 429
    verbose=True
)


Saving 1m shards to: data/minute
[minute] BTCUSDT: downloading 1m from 2023-10-30 to 2025-10-30 ...
[fetch] BTCUSDT 1m 202310: 2023-10-30 → 2023-11-01
  -> rows=2,881 saved: data/minute/binance_btcusdt_1m_202310.parquet
[fetch] BTCUSDT 1m 202311: 2023-11-01 → 2023-12-01
  -> rows=43,201 saved: data/minute/binance_btcusdt_1m_202311.parquet
[fetch] BTCUSDT 1m 202312: 2023-12-01 → 2024-01-01
  -> rows=44,641 saved: data/minute/binance_btcusdt_1m_202312.parquet
[fetch] BTCUSDT 1m 202401: 2024-01-01 → 2024-02-01
  -> rows=44,641 saved: data/minute/binance_btcusdt_1m_202401.parquet
[fetch] BTCUSDT 1m 202402: 2024-02-01 → 2024-03-01
  -> rows=41,761 saved: data/minute/binance_btcusdt_1m_202402.parquet
[fetch] BTCUSDT 1m 202403: 2024-03-01 → 2024-04-01
  -> rows=44,641 saved: data/minute/binance_btcusdt_1m_202403.parquet
[fetch] BTCUSDT 1m 202404: 2024-04-01 → 2024-05-01
  -> rows=43,201 saved: data/minute/binance_btcusdt_1m_202404.parquet
[fetch] BTCUSDT 1m 202405: 2024-05-01 → 2024-06-01
  -

In [6]:
# Build derived 1h-from-1m features for your symbols
try:
    from features_minute import build_hour_from_minute, default_params, make_feature_set_id
except Exception:
    import sys, os
    sys.path.append("simplest_ml/w1")
    from features_minute import build_hour_from_minute, default_params, make_feature_set_id

symbols = ["BTCUSDT","BNBUSDT","SOLUSDT","ETHUSDT","AVAXUSDT","TRXUSDT","ARBUSDT","SUIUSDT"]
minute_dir   = "data/minute"   # or "data/minute" if running inside w1
derived_dir  = "data/derived"  # where derived hourly shards go
params = default_params()
fid = make_feature_set_id(params)
for sym in symbols:
    build_hour_from_minute(
        symbol=sym,
        start="2023-10-30",
        end="2025-10-30",
        params=params,
        minute_dir=minute_dir,
        derived_dir=derived_dir,
        overwrite=False,
        verbose=True,
    )
print("feature_set_id:", fid)


[keep] BTCUSDT 202310: data/derived/btcusdt_hour_from_1m_v4m_{"bipower_window_min":60,"ret_windows_min":[5,15,60],"skew_kurt_windows_min":[15,60],"vol_windows_min":[15,60],"volsum_windows_min":[5,15,60],"vwap_window_min":60,"z_windows_hours":[48,168]}_202310.parquet
[keep] BTCUSDT 202311: data/derived/btcusdt_hour_from_1m_v4m_{"bipower_window_min":60,"ret_windows_min":[5,15,60],"skew_kurt_windows_min":[15,60],"vol_windows_min":[15,60],"volsum_windows_min":[5,15,60],"vwap_window_min":60,"z_windows_hours":[48,168]}_202311.parquet
[keep] BTCUSDT 202312: data/derived/btcusdt_hour_from_1m_v4m_{"bipower_window_min":60,"ret_windows_min":[5,15,60],"skew_kurt_windows_min":[15,60],"vol_windows_min":[15,60],"volsum_windows_min":[5,15,60],"vwap_window_min":60,"z_windows_hours":[48,168]}_202312.parquet
[keep] BTCUSDT 202401: data/derived/btcusdt_hour_from_1m_v4m_{"bipower_window_min":60,"ret_windows_min":[5,15,60],"skew_kurt_windows_min":[15,60],"vol_windows_min":[15,60],"volsum_windows_min":[5,15,